# Nonexistence of a $[\![7,1,4]\!]_4$ stabiliser code

A $[\![7,1,4]\!]_4$ code is equivalent to a set of seven solids (projective 3-spaces) in PG(11,2), each equipped with a symplectic polarity, such that every three solids span the entire space, and such that every codimension two space intersects an even number of solids in a hyperbolic line.

If we project from one of the solids, we get a $[\![6,2,3]\!]_4$ code, or equivalently, six solids in PG(7,2), each equipped with a symplectic polarity, such that every two solids span the entire space, and such that every codimension two space intersects an even number of solids in a hyperbolic line. We therefore start by generating all arcs of six solids in PG(7,2).

## Arcs of six solids in PG(7,2)

We first show that there are 341 configurations of six solids in PG(7,2) such that every two solids span the entire space (i.e. an arc of six solids). Without loss of generality, the four solids are spanned by the four column blocks of a block matrix $$G=\begin{pmatrix}I&O&I&I&I&I\\O&I&I&A&B&C\end{pmatrix}$$ where every two column blocks have full rank, and $A,B,C$ are determined up to conjugation and permutation.

In [1]:
O = zero_matrix(GF(2),4)
I = identity_matrix(GF(2),4)

With the purpose of storing them as keys in a dictionary, the following funtion converts matrices into tuples:

In [2]:
def tuplify(M): return tuple(tuple(row) for row in M) # makes matrices hashable

The following function does row operations to bring a block matrix in "standard form", where the first column block is spanned by $\begin{pmatrix}I\\O\end{pmatrix}$, the second one by $\begin{pmatrix}O\\I\end{pmatrix}$ and the third one by $\begin{pmatrix}I\\I\end{pmatrix}$.

In [3]:
def standard_form(G):
    L1 = ~G[:,:8]
    G = L1*G # row-reduced form
    L2 = block_diagonal_matrix(I,G[:4,8:12]*~G[4:,8:12])
    G = L2*G # third column block is now spanned by [[I],[I]]
    return G, L2*L1 # we multiplied with L2*L1 on the left to bring into standard form

For any block $A$ (resp. $B,C$), the submatrices $\begin{pmatrix}I&I\\O&A\end{pmatrix}$ and $\begin{pmatrix}I&I\\I&A\end{pmatrix}$ should be invertible, i.e. $A$ and $A+I$ should be invertible.

In [4]:
GL = [M for M in MatrixSpace(GF(2),4,4) if M.is_invertible()]
blocks = [M for M in GL if (M+I).is_invertible()]

### Arcs of four solids in PG(7,2)

First, we determine all $A$'s up to conjugation.

In [5]:
conj_representatives_A = [] # A-blocks up to conjugation
get_conj_representative_A = {} # stores the conjugation representative of the A-block
for A in blocks:
    a = tuplify(A)
    if not a in get_conj_representative_A: # conjugacy class not yet considered
        conj_representatives_A.append(a)
        for X in GL:
            a_equiv = tuplify(~X*A*X)
            if not a_equiv in get_conj_representative_A:
                get_conj_representative_A[a_equiv] = a, X # conjugate with X to get A
len(conj_representatives_A)

5

Now also up to permutation.

In [6]:
representatives_A = [] # A-blocks up to equivalence
get_representative_A = {} # stores the representative of the A-block
for a, (a_conj, X) in get_conj_representative_A.items():
    if not a_conj in get_representative_A: # permutation class not yet considered
        representatives_A.append(a_conj)
        G = block_matrix(GF(2),[[1,0,1,1],[0,1,1,matrix(a_conj)]])
        columns = [G[:,4*i:4*i+4] for i in range(4)] # column blocks of G
        for pi in Permutations(4):
            Gnew, L = standard_form(block_matrix(GF(2), [[columns[pi(i+1)-1] for i in range(4)]])) # permuted and put in standard form
            a_equiv, Y = get_conj_representative_A[tuplify(Gnew[4:,12:]*~Gnew[:4,12:])] # conjugation representative of the new A-block
            if not a_equiv in get_representative_A:
                get_representative_A[a_equiv] = a_conj, ~L*~block_diagonal_matrix(Y,Y) # multiply with this on the left to get A
    a_perm, L = get_representative_A[a_conj]
    get_representative_A[a] = a_perm, L*block_diagonal_matrix(X,X) # multiply with this on the left to get A_perm
len(representatives_A) # number of configurations of four solids such that every two span the whole space

3

### Arcs of five solids in PG(7,2)

Next, we determine all $A$'s and $B$'s up to equivalence. First, up to conjugation.

In [7]:
conj_representatives_AB = [] # (A,B)-blocks up to conjugation
get_conj_representative_AB = {} # stores the conjugation representative of the (A,B)-block
centraliser = {} # stores the invertible matrices that commute with A
compatibleblocks = {} # stores the matrices B (or C) that exist as a block together with A
for a in representatives_A:
    A = matrix(a)
    centraliser[a] = [X for X in GL if A*X==X*A]
    compatibleblocks[a] = [B for B in blocks if (A+B).is_invertible()]
    for B in compatibleblocks[a]:
        b = tuplify(B)
        if not (a,b) in get_conj_representative_AB: # conjugacy class not yet considered
            conj_representatives_AB.append((a,b))
            for X in centraliser[a]:
                b_equiv = tuplify(~X*B*X)
                if not (a,b_equiv) in get_conj_representative_AB:
                    get_conj_representative_AB[(a,b_equiv)] = b, X # conjugate with X to get (A,B)
len(conj_representatives_AB)

305

Up to permutation:

In [8]:
representatives_AB = [] # (A,B)-blocks up to equivalence
get_representative_AB = {} # stores the representative of the (A,B)-block
for (a,b), (b_conj, X) in get_conj_representative_AB.items():
    if not (a,b_conj) in get_representative_AB: # permutation class not yet considered
        representatives_AB.append((a,b_conj))
        G = block_matrix(GF(2),[[1,0,1,1,1],[0,1,1,matrix(a),matrix(b_conj)]])
        columns = [G[:,4*i:4*i+4] for i in range(5)] # column blocks of G
        for pi in Permutations(5):
            Gnew, L1 = standard_form(block_matrix(GF(2), [[columns[pi(i+1)-1] for i in range(5)]])) # permuted and put in standard form
            a_equiv, L2 = get_representative_A[tuplify(Gnew[4:,12:16]*~Gnew[:4,12:16])] # representative of the new A-block
            Gnew = L2*Gnew
            b_equiv, Y = get_conj_representative_AB[(a_equiv,tuplify(Gnew[4:,16:]*~Gnew[:4,16:]))] # representative of the new (A,B)-block
            if not (a_equiv,b_equiv) in get_representative_AB:
                get_representative_AB[(a_equiv,b_equiv)] = a,b_conj, ~L1*~L2*~block_diagonal_matrix(Y,Y) # multiply with this on the left to get (A,B)
    a_perm,b_perm, L = get_representative_AB[(a,b_conj)]
    get_representative_AB[(a,b)] = a_perm,b_perm, L*block_diagonal_matrix(X,X) # multiply with this on the left to get (A_perm,B_perm)
len(representatives_AB) # number of configurations of five solids such that every two span the whole space

22

### Arcs of six solids in PG(7,2)

Similarly, we determine all nonequivalent triples $(A,B,C)$.

In [9]:
conj_representatives_ABC = [] # (A,B,C)-blocks up to conjugation
get_conj_representative_ABC = {} # stores the representative of the (A,B,C)-block
for a,b in representatives_AB:
    A,B = matrix(a),matrix(b)
    centraliser_AB = [X for X in centraliser[a] if B*X==X*B]
    for C in compatibleblocks[a]:
        c = tuplify(C)
        if (B+C).is_invertible() and not (a,b,c) in get_conj_representative_ABC: # conjugacy class not yet considered
            conj_representatives_ABC.append((a,b,c))
            for X in centraliser_AB:
                c_equiv = tuplify(~X*C*X)
                if not (a,b,c_equiv) in get_conj_representative_ABC:
                    get_conj_representative_ABC[(a,b,c_equiv)] = c, X # conjugate with X to get (A,B,C)
len(conj_representatives_ABC)

5830

In [10]:
# may take a few minutes
representatives_ABC = [] # (A,B,C)-blocks up to equivalence
get_representative_ABC = {} # stores the representative of the (A,B,C)-block
for (a,b,c), (c_conj, X) in get_conj_representative_ABC.items():
    if not (a,b,c_conj) in get_representative_ABC: # permutation class not yet considered
        representatives_ABC.append((a,b,c_conj))
        G = block_matrix(GF(2),[[1,0,1,1,1,1],[0,1,1,matrix(a),matrix(b),matrix(c_conj)]])
        columns = [G[:,4*i:4*i+4] for i in range(6)] # column blocks of G
        for pi in Permutations(6):
            Gnew, L1 = standard_form(block_matrix(GF(2), [[columns[pi(i+1)-1] for i in range(6)]])) # permuted and put in standard form
            a_equiv, L2 = get_representative_A[tuplify(Gnew[4:,12:16]*~Gnew[:4,12:16])] # representative of the new A-block
            Gnew = L2*Gnew
            a_equiv,b_equiv, L3 = get_representative_AB[(a_equiv,tuplify(Gnew[4:,16:20]*~Gnew[:4,16:20]))] # representative of the new (A,B)-block
            Gnew = L3*Gnew
            c_equiv, Y = get_conj_representative_ABC[(a_equiv,b_equiv,tuplify(Gnew[4:,20:]*~Gnew[:4,20:]))] # representative of the new (A,B,C)-block
            if not (a_equiv,b_equiv,c_equiv) in get_representative_ABC:
                    get_representative_ABC[(a_equiv,b_equiv,c_equiv)] = a,b,c_conj
    a_perm,b_perm,c_perm = get_representative_ABC[(a,b,c_conj)]
    get_representative_ABC[(a,b,c)] = a_perm,b_perm,c_perm
len(representatives_ABC) # number of configurations of six solids such that every two span the whole space

341

So there are 341 different arcs of six solids in PG(7,2).

## $[\![6,2,3]\!]_4$ stabiliser codes

For each of the 341 configurations of six solids, we determine all possible symplectic polarities on the solids such that every codimension two space intersects an even number of solids in a hyperbolic line. Suppose that this is the case. For each line $\ell$ in each of the six solids (so $6\cdot35=210$ lines), let $$x_\ell=\begin{cases}0\text{ if \(\ell\) is totally isotropic}\\1\text{ if \(\ell\) is hyperbolic}\end{cases}.$$
Then for each codimension two space $\pi$, the sum of the $x_\ell$'s for which $\ell$ is the intersection of $\pi$ with one of the six solids, equals zero modulo 2. So it suffices to check a system of homogeneous equations over $\mathbb{F}_2$, which we calculate as the nullspace of a matrix M.

In [11]:
V = VectorSpace(GF(2),8)
codim2s = list(V.subspaces(6))
len(codim2s)

10795

The first three solids are always the same, so we can do some precalculations for all cases.

In [12]:
solids = [V.subspace(list(block_matrix(GF(2),[x]))) for x in [[I,O],[O,I],[I,I]]]
label = {} # stores the label of a line
i = 0
for solid in solids:
    for line in solid.subspaces(2):
        label[line] = i
        i += 1
M = [] # 10795 x 210 matrix
for pi in codim2s:
    row = [0]*210
    for solid in solids:
        intersection = pi.intersection(solid)
        if intersection.dimension() == 2:
            row[label[intersection]] = 1
    M.append(row)

Given such a solution, one still has to check whether the lines $\ell$ for which $x_\ell=1$, really are hyperbolic lines of a symplectic polarity. W(3,2) contains 15 totally isotropic lines and 20 hyperbolic lines, so we first of all check that $|\{\ell\in\pi\mid x_\ell=1\}|=20$. Then, we choose a (potential) hyperbolic line $\ell_1$ and loop over all (potential) hyperbolic lines $\ell_2$ to check whether $\ell_2=\ell_1^\perp$ (we can fix $\ell_1$ by Witt's theorem). This is true if and only if every other (potential) hyperbolic line intersects exactly one of them. If we find such a line $\ell_2$, then these lines are indeed the hyperbolic lines of a symplectic polarity.

These solutions, that are indeed the hyperbolic lines of symplectic polarities, are then stored as a "stabiliser matrix" $$G=\begin{pmatrix}X_0&O&Y&A_0&B_0&C_0\\O&X_1&Y&A_1&B_1&C_1\end{pmatrix}$$ of a $[\![6,2,3]\!]_4$ code, where the first two columns of every column block span a hyperbolic line $\ell$ and the next two span $\ell^\perp$.

In [13]:
# may take a few hours
count = 0
stabs = [] # stabiliser matrices of [[6,2,3]]_4 codes
for a,b,c in representatives_ABC:
    count += 1
    print("Calculating solutions for configuration nr. " + str(count) + ".")
    A,B,C = matrix(a),matrix(b),matrix(c)
    solids = [V.subspace(list(block_matrix(GF(2),[x]))) for x in [[I,O],[O,I],[I,I],[I,A.transpose()],[I,B.transpose()],[I,C.transpose()]]]
    i = 105 # continuing from the fourth solid
    for solid in solids[3:]:
        for line in solid.subspaces(2):
            label[line] = i
            i += 1
    row = 0
    for pi in codim2s:
        for i in range(105,210):
            M[row][i] = 0 # reset the coefficients to zero
        for solid in solids[3:]:
            intersection = pi.intersection(solid)
            if intersection.dimension() == 2:
                M[row][label[intersection]] = 1
        row += 1
    for v in matrix(GF(2), M).right_kernel():
        v = list(v)
        if not all([v[35*i:35*(i+1)].count(1)==20 for i in range(6)]):
            continue
        G = matrix(ncols=8,nrows=0) # transposed "stabiliser matrix", every two rows span a line
        for solid in solids:
            symplectic_lines = [line for line in solid.subspaces(2) if v[label[line]]==1]
            l1 = symplectic_lines[0] # w.l.o.g. by Witt's theorem
            for l2 in symplectic_lines:
                symplectic = all(line.intersection(l1).dimension() != line.intersection(l2).dimension() for line in symplectic_lines) # if l2 = perp(l1)
                if symplectic:
                    G = G.stack(matrix(l1.basis()))
                    G = G.stack(matrix(l2.basis()))
                    break
            if not symplectic:
                break
        if symplectic:
            stabs.append(G.transpose())
len(stabs)

Calculating solutions for configuration nr. 1.
Calculating solutions for configuration nr. 2.
Calculating solutions for configuration nr. 3.
Calculating solutions for configuration nr. 4.
Calculating solutions for configuration nr. 5.
Calculating solutions for configuration nr. 6.
Calculating solutions for configuration nr. 7.
Calculating solutions for configuration nr. 8.
Calculating solutions for configuration nr. 9.
Calculating solutions for configuration nr. 10.
Calculating solutions for configuration nr. 11.
Calculating solutions for configuration nr. 12.
Calculating solutions for configuration nr. 13.
Calculating solutions for configuration nr. 14.
Calculating solutions for configuration nr. 15.
Calculating solutions for configuration nr. 16.
Calculating solutions for configuration nr. 17.
Calculating solutions for configuration nr. 18.
Calculating solutions for configuration nr. 19.
Calculating solutions for configuration nr. 20.
Calculating solutions for configuration nr. 21.
C

1311

So there are at most 1311 different $[\![6,2,3]\!]_4$ codes.

## None of the $[\![6,2,3]\!]_4$ codes extends to a $[\![7,1,4]\!]_4$ code

Up to equivalence, the stabiliser matrix of a $[\![7,1,4]\!]_4$ code looks like $$\begin{pmatrix}X_0&O&O&Y&A_0&B_0&C_0\\O&X_1&O&Y&A_1&B_1&C_1\\O&O&X_2&Y&A_2&B_2&C_2\end{pmatrix}$$ where $$\begin{pmatrix}X_0&O&Y&A_0&B_0&C_0\\O&X_1&Y&A_1&B_1&C_1\end{pmatrix}$$ is the "stabiliser matrix" of a $[\![6,2,3]\!]_4$ code (one of the 1311 previous solutions), where every two columns span a hyperbolic line and its perpendicular line. It has the property that any two rows are symplectic orthogonal, and any three column blocks have full rank. The following function checks the latter property for a given matrix with seven column blocks of size four:

In [14]:
def is_arc(G):
    columns = [G[:,4*i:4*i+4] for i in range(7)] # column blocks of G
    for S in Subsets(range(7),3):
        C1,C2,C3 = columns[S[0]],columns[S[1]],columns[S[2]]
        if not rank(block_matrix([[C1,C2,C3]])) == G.nrows():
            return False
    return True

Given a solution for the stabiliser matrix of a $[\![6,2,3]\!]_4$ code, we first determine all possible submatrices $$\begin{pmatrix}Y&A_2&B_2&C_2\end{pmatrix}$$ row by row. Each of these rows is symplectic orthogonal to the rows of $$\begin{pmatrix}Y&A_0&B_0&C_0\\Y&A_1&B_1&C_1\end{pmatrix}$$ with respect to the symplectic form
$$f(v,w)=v_1w_2+v_2w_1+\cdots+v_{15}w_{16}+v_{16}w_{15}.$$

In [16]:
# takes about 15 minutes
for G in stabs:
    M = G[:,8:] # from now on, we are only interested in the last four column blocks
    Y = M[:4,:4]
    A0 = M[:4,4:8]
    A1 = M[4:,4:8]
    B0 = M[:4,8:12]
    B1 = M[4:,8:12]
    C0 = M[:4,12:]
    C1 = M[4:,12:]
    R0 = block_matrix([[I,O,O,Y,A0,B0,C0]])
    R1 = block_matrix([[O,I,O,Y,A1,B1,C1]])
    W = identity_matrix(GF(2),8).tensor_product(matrix(GF(2),[[0,1,],[1,0]])) # symplectic form
    ortho = (M*W).right_kernel() # rows that are orthogonal to those of M
    appendable_rows = {i:[r for r in ortho if r[:4]==vector(Y[i:i+1,:])] for i in range(4)}
    for r0 in appendable_rows[0]:
        full_r0 = matrix([0,0,0,0,0,0,0,0,1,0,0,0]+list(r0))
        if not is_arc(block_matrix([[R0],[R1],[full_r0]])):
            continue
        for r1 in appendable_rows[1]:
            full_r1 = matrix([0,0,0,0,0,0,0,0,0,1,0,0]+list(r1))
            if not is_arc(block_matrix([[R0],[R1],[full_r0],[full_r1]])):
                continue
            for r2 in appendable_rows[2]:
                full_r2 = matrix([0,0,0,0,0,0,0,0,0,0,1,0]+list(r2))
                if not is_arc(block_matrix([[R0],[R1],[full_r0],[full_r1],[full_r2]])):
                    continue
                for r3 in appendable_rows[3]:
                    full_r3 = matrix([0,0,0,0,0,0,0,0,0,0,0,1]+list(r3))
                    if not is_arc(block_matrix([[R0],[R1],[full_r0],[full_r1],[full_r2],[full_r3]])):
                        continue
                    rows = [r0,r1,r2,r3]
                    for r in rows:
                        for s in rows:
                            if r*W*s != 0:
                                print("A [[7,1,4]]_4 code might exist.") # if the symplectic inner product of every two of these rows is zero, a [[7,1,4]]_4 code does not exist
print("Done.")

Done.


We observed that the rows of $\begin{pmatrix}Y&A_2&B_2&C_2\end{pmatrix}$ are always pairwise symplectic orthogonal. Hence the rows of $X_2$ should also be pairwise orthogonal, but $X_2$ has full rank, a contradiction. Thus, a $[\![7,1,4]\!]_4$ code does not exist. As a corollary, an $[\![8,0,5]\!]_4$ code does not exist either.